In [1]:
# Init Python

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

print("Init Python")

import sys
import logging

# Add Subdirectory with custom Python source modules to path
sys.path.append("src")

# Log Level
# log_level = logging.INFO
log_level = logging.DEBUG

logger_name = "root"

logging.basicConfig(stream=sys.stdout, level=log_level)
log = logging.getLogger(logger_name)
log.setLevel(log_level)

http_log = logging.getLogger("urllib3.connectionpool")
http_log.setLevel(logging.INFO)

log.info("Successfully initialized Python")

Init Python
INFO:root:Successfully initialized Python


In [2]:
import csv
import pandas as pd
import numpy as np
import requests
import datetime
from bs4 import BeautifulSoup

company = 'Example'

log.info("Download Google product categories")
url = 'http://www.google.com/basepages/producttype/taxonomy-with-ids.en-US.txt'
categoryfile = requests.get(url, allow_redirects=True)
open('data/taxonomy-with-ids.en-US.txt', 'wb').write(categoryfile.content)
with open("data/taxonomy-with-ids.en-US.txt", "rt") as fin:
    with open("data/taxonomy-with-ids.en-US.csv", "wt") as fout:
        for line in fin:
            fout.write(line.replace('# Google_Product_Taxonomy_Version: 2021-09-21','"google_category_id","google_category_name').replace(' - ', ',"').splitlines()[0] + '"\n')

log.info("Loading custom category mappings from CSV file")
google_category_mapping = pd.read_csv('input/google_category_mapping.csv', error_bad_lines=False, dtype = {
    'category': str,
    'sub_category': str,
    'sub_sub_category': str,
    'google_category_id': str,
    'google_category_name': str
})

log.info("Loading product CSV file(s)")
def read_csv(filelocation):
    return pd.read_csv(filelocation, encoding = "cp1252", error_bad_lines=False, dtype = {
        'SKU': str,
        'Title': str,
        'Description': str,
        'Detailed Description': str,
        'Thumbnail Image': str,
        'Product Image': str,
        'Price': np.float64,
        'Wholesale Price': str,
        'Weight (Kgs)': str,
        'Volume': np.int64,
        'Inactive': str,
        'Large Image': str,
        'GTIN': str,
        'Brand': str,
        'Manufacturer Part Number': str,
        'Product Page Title': str,
        'Product Keywords': str,
        'Product Page Description': str,
        'product_type': str,
        'Extended Code Field': str,
        'URL Override': str,
        'URL Slug': str,
        'Last Modification Date': str,
        'Supplier': str,
        'Custom Data 1': str,
        'Custom Data 2': str,
        'Custom Data 3': str,
        'Custom Data 4': str,
        'FAQs': str,
        'External Id': str,
        'frequency': str,
        'Quantity': str,
        'Stock Message': str,
        'Promotion Discount': str,
        'Promotion Type': str,
        'Promotion Text': str,
        'Promotion Timed': str,
        'Promotion Start Date': str,
        'Promotion End Date': str,
        'Product Tags': str,
        'Product Attributes': str,
        'Category': str,
        'Sub category': str,
        'Subx2 category': str,
        'Subx3 category': str,
        'Subx4 category': str
    })

page1 = read_csv('input/products_export_1-2000.csv')
page2 = read_csv('input/products_export_2001-4000.csv')
df1 = pd.concat([page1, page2])

log.info("Filter by active")
df2 = df1[df1['Inactive'] == '0']
df2 = df2[df2['Price'] != '0.00']

log.info("Use only first category")
def map_reduce_to_first_cat(row):
    lines = str(row['Category']).splitlines()
    if(len(lines) > 1):
        return lines[0]
    return row['Category']

df2['Category'] = df2.apply(map_reduce_to_first_cat, axis=1)
def map_reduce_to_first_sub_cat(row):
    lines = str(row['Sub category']).splitlines()
    if(len(lines) > 1):
        return lines[0]
    return row['Sub category']

df2['Sub category'] = df2.apply(map_reduce_to_first_sub_cat, axis=1)

def map_reduce_to_first_sub_sub_cat(row):
    lines = str(row['Subx2 category']).splitlines()
    if(len(lines) > 1):
        return lines[0]
    return row['Subx2 category']

df2['Subx2 category'] = df2.apply(map_reduce_to_first_sub_sub_cat, axis=1)

log.info("Resolve Google category")
def map_google_category(row):
    cat_mapping = google_category_mapping[google_category_mapping['category'] == row['Category']]
    if(cat_mapping.empty):
        return google_category_mapping[google_category_mapping['category'] == 'DEFAULT']['google_category_name'].iloc[0]
    else:
        if(len(cat_mapping.index) == 1):
            return cat_mapping['google_category_name'].iloc[0]
        else:
            sub_cat_mapping = cat_mapping[cat_mapping['sub_category'] == row['Sub category']]
            if(sub_cat_mapping.empty):
                return cat_mapping['google_category_name'].iloc[0]
            else:
                if(len(sub_cat_mapping) == 1):
                    return sub_cat_mapping['google_category_name'].iloc[0]
                else:
                    sub_sub_cat_mapping = cat_mapping[cat_mapping['sub_sub_category'] == row['Subx2 category']]
                    if(sub_sub_cat_mapping.empty):
                        return sub_cat_mapping['google_category_name'].iloc[0]
                    else:
                        if(len(sub_sub_cat_mapping) == 1):
                            return sub_sub_cat_mapping['google_category_name'].iloc[0]

df2['google_product_category'] = df2.apply(map_google_category, axis=1)

log.info("Aggregate category to Google product type")
def map_google_product_type(row):
    cat = str(row['Category']).replace('\r', '').replace('\n', '')
    if(not str(row['Sub category']).replace('\r', '').replace('\n', '') == '' and not pd.isnull(row['Sub category'])):
        cat = cat + ' > ' + str(row['Sub category']).replace('\r', '').replace('\n', '')
        if(not str(row['Subx2 category']).replace('\r', '').replace('\n', '') == '' and not pd.isnull(row['Subx2 category'])):
            cat = cat + ' > ' + str(row['Subx2 category']).replace('\r', '').replace('\n', '')
            if(not str(row['Subx3 category']).replace('\r', '').replace('\n', '') == '' and not pd.isnull(row['Subx3 category'])):
                cat = cat + ' > ' + str(row['Subx3 category']).replace('\r', '').replace('\n', '')
                if(not str(row['Subx4 category']).replace('\r', '').replace('\n', '') == '' and not pd.isnull(row['Subx4 category'])):
                    cat = cat + ' > ' + str(row['Subx4 category']).replace('\r', '').replace('\n', '')
    return cat

df2['google_product_type'] = df2.apply(map_google_product_type, axis=1)

log.info("Full Image URL")
def map_image_link(row):
    if(pd.isnull(row['Large Image']) or str(row['Large Image']).replace('\r', '').replace('\n', '') == ''):
        return ''
    else:
        return str('https://www.' + company.lower() + '.co.nz/site/' + company.lower() + '/images/large/') + str(row['Large Image'])

df2['image_link'] = df2.apply(map_image_link, axis=1)

log.info("Rename Columns")
df2 = df2.rename(columns={'Title': 'title',
                    'Description': 'short_description',
                    'Detailed Description': 'description',
                    'Brand': 'brand',
                   })
df3 = df2

log.info("Add additional columns")
df3['condition'] = 'new'
def map_mpn(row):
    return row['SKU'] if row['Manufacturer Part Number'] == '' else row['Manufacturer Part Number']

df3['mpn'] = df3.apply(map_mpn, axis=1)
df3['identifier_exists'] = 'no'
def map_availability1(row):
    return 'out_of_stock' if row['Quantity'] == '0' else 'in_stock'

df3['availability'] = df3.apply(map_availability1, axis=1)

def map_price(row):
    return str(row['Price']) + ' NZD'

df3['price'] = df3.apply(map_price, axis=1)
def map_link2(row):
    return 'https://www.' + company.lower() + '.co.nz/product?sku=' + str(row['SKU'])

df3['link'] = df3.apply(map_link2, axis=1)
def map_brand(row):
    return company + ' Machine Tools' if row['brand'] == '' else row['brand']

df3['brand'] = df3.apply(map_brand, axis=1)
def map_sale_price(row):
    if(pd.isnull(row['Promotion Discount']) or str(row['Promotion Discount']).replace('\r', '').replace('\n', '') == '' or str(row['Promotion Discount']).replace('\r', '').replace('\n', '') == '0'):
        return ''
    else:
        if(pd.isnull(row['Promotion End Date']) or str(row['Promotion End Date']).replace('\r', '').replace('\n', '') == '' or str(row['Promotion End Date']).replace('\r', '').replace('\n', '') == '0'):
            return ''
        else:
            try:
                promotion_start_date = datetime.datetime.strptime(str(row['Promotion Start Date']).replace('\r', '').replace('\n', ''), '%Y%m%d')
                promotion_end_date = datetime.datetime.strptime(str(row['Promotion End Date']).replace('\r', '').replace('\n', ''), '%Y%m%d')
                now_date = datetime.datetime.now()
                if(now_date < promotion_end_date or promotion_start_date == promotion_end_date):
                    if '%' in str(row['Promotion Discount']):
                        return str(float(row['Price']) - (float(row['Price']) * (float(str(row['Promotion Discount']).replace('%', ''))/100))) + ' NZD'
                    elif str(row['Promotion Discount']).isnumeric():
                        return str(float(row['Price']) - float(row['Promotion Discount'])) + ' NZD'
                    else:
                        log.warning('Invalid Promotion Discount ' +  str(row['Promotion Discount']) + ' for ' + row['SKU'])
                        return ''
                else:
                    return ''
            except ValueError:
                promotion_start_date = datetime.datetime.strptime(str(row['Promotion Start Date']).replace('\r', '').replace('\n', '').replace('31', '30'), '%Y%m%d')
                promotion_end_date = datetime.datetime.strptime(str(row['Promotion End Date']).replace('\r', '').replace('\n', '').replace('31', '30'), '%Y%m%d')
                now_date = datetime.datetime.now()
                if(now_date < promotion_end_date or promotion_start_date == promotion_end_date):
                    if '%' in str(row['Promotion Discount']):
                        return str(float(row['Price']) - (float(row['Price']) * (float(str(row['Promotion Discount']).replace('%', ''))/100))) + ' NZD'
                    elif str(row['Promotion Discount']).isnumeric():
                        return str(float(row['Price']) - float(row['Promotion Discount'])) + ' NZD'
                    else:
                        log.warning('Invalid Promotion Discount ' +  str(row['Promotion Discount']) + ' for ' + row['SKU'])
                        return ''
                else:
                    return ''

df3['sale_price'] = df3.apply(map_sale_price, axis=1)
def map_sale_price_effective_date(row):
    if(pd.isnull(row['sale_price']) or str(row['sale_price']).replace('\r', '').replace('\n', '') == '' or str(row['sale_price']).replace('\r', '').replace('\n', '') == '0'):
        return ''
    else:
        try:
            promotion_start_date = datetime.datetime.strptime(str(row['Promotion Start Date']).replace('\r', '').replace('\n', ''), '%Y%m%d')
            promotion_end_date = datetime.datetime.strptime(str(row['Promotion End Date']).replace('\r', '').replace('\n', ''), '%Y%m%d')
            if (promotion_start_date == promotion_end_date):
                return promotion_start_date.strftime('%Y-%m-%d') + 'T00:00Z/2050-12-31T23:59Z'
            else:
                return promotion_start_date.strftime('%Y-%m-%d') + 'T00:00Z/' + promotion_end_date.strftime('%Y-%m-%d') + 'T23:59Z'
        except:
#             log.warn('Invalid Promotion date ' + str(row['Promotion End Date']) + ' for ' + row['SKU'])
            promotion_start_date = datetime.datetime.strptime(str(row['Promotion Start Date']).replace('\r', '').replace('\n', '').replace('31', '30'), '%Y%m%d')
            promotion_end_date = datetime.datetime.strptime(str(row['Promotion End Date']).replace('\r', '').replace('\n', '').replace('31', '30'), '%Y%m%d')
            if (promotion_start_date == promotion_end_date):
                return promotion_start_date.strftime('%Y-%m-%d') + 'T00:00Z/2050-12-31T23:59Z'
            else:
                return promotion_start_date.strftime('%Y-%m-%d') + 'T00:00Z/' + promotion_end_date.strftime('%Y-%m-%d') + 'T23:59Z'

df3['sale_price_effective_date'] = df3.apply(map_sale_price_effective_date, axis=1)
df3['gender'] = 'unisex'
df3['age_group'] = 'adult'
def map_shipping_weight(row):
    if(pd.isnull(row['Weight (Kgs)']) or str(row['Weight (Kgs)']).replace('\r', '').replace('\n', '') == ''):
        return ''
    else:
        return str(row['Weight (Kgs)']) + ' kg'

df3['shipping_weight'] = df3.apply(map_shipping_weight, axis=1)

def map_html(row):
    if(pd.isnull(row['description']) or str(row['description']).replace('\r', '').replace('\n', '') == '' or str(row['description']).replace('\r', '').replace('\n', '') == '0'):
        return ''
    # Un-HTML
    soup = BeautifulSoup(row['description'])
    print(soup.get_text())
    return soup.get_text()

df3['description'] = df3.apply(map_html, axis=1)

# df3['adult'] = str('false')
df3['adult'] = 'no'

# Empty columns
# df3['gtin'] = ''
# df3['promotion_id'] = ''
# df3['item_group_id'] = ''
# df3['is_bundle'] = ''
# df3['custom_label_0'] = ''
# df3['custom_label_1'] = ''
# df3['shipping_label'] = ''
# df3['size'] = ''
# df3['color'] = ''
# df3['size_type'] = ''
# df3['additional_image_link'] = ''
# df3['shipping_length'] = ''
# df3['shipping_height'] = ''
# df3['shipping_width'] = ''


log.info("Remove unused Columns")
df3 = df3.drop(columns=['short_description',
                        'Thumbnail Image',
                        'Product Image',
                        'Price',
                        'Wholesale Price',
                        'Weight (Kgs)',
                        'Volume',
                        'Inactive',
                        'Large Image',
                        'GTIN',
                        'Manufacturer Part Number',
                        'Product Page Title',
                        'Product Keywords',
                        'Product Page Description',
                        'product_type',
                        'Extended Code Field',
                        'URL Override',
                        'URL Slug',
                        'Last Modification Date',
                        'Supplier',
                        'Custom Data 1',
                        'Custom Data 2',
                        'Custom Data 3',
                        'Custom Data 4',
                        'FAQs',
                        'External Id',
                        'frequency',
                        'Quantity',
                        'Stock Message',
                        'Promotion Discount',
                        'Promotion Type',
                        'Promotion Text',
                        'Promotion Timed',
                        'Promotion Start Date',
                        'Promotion End Date',
                        'Product Tags',
                        'Product Attributes',
                        'Category',
                        'Sub category',
                        'Subx2 category',
                        'Subx3 category',
                        'Subx4 category'])
df3 = df3.rename(columns={'SKU': 'id', 'google_product_type': 'product_type'})

df3.to_csv('output/products_google.csv', index=None)
df3.to_csv('output/products_facebook.csv', index=None)

INFO:root:Download Google product categories
INFO:root:Loading custom category mappings from CSV file
INFO:root:Loading product CSV file(s)
INFO:root:Filter by active
INFO:root:Use only first category
INFO:root:Resolve Google category


/Users/thomas/.pyenv/versions/3.7.4/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/thomas/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:79: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


/Users/thomas/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:80: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.




INFO:root:Aggregate category to Google product type
INFO:root:Full Image URL
INFO:root:Rename Columns
INFO:root:Add additional columns
Two component acrylic HS clearcoat 2:1. High quality, scratch resistant clear suitable for partial and total respray spot panel repairs. Easy to apply. High gloss finish. VOC compliant.7.5L Kit includes: EN Chemicals 5800 HS Acrylic Clearcoat 5 Litre #EN58005L EN Chemicals Fast Hardener 2.5 Litre #EN620025L or EN Chemicals Normal Hardener 2.5 Litre #EN61002 to 1 Mixing Ratio.Features: High gloss Scratch resistant VOC compliant Downloads for:   			EN5800			Safety Data Sheet (558KB)   			EN6100 Hardener Normal			Safety Data Sheet (476KB) 			EN6200 Hardener Fast			Safety Data Sheet (476KB)  
A two component primer surfacer that can be applied direct to metal surfaces. It provides corrosion protection along with excellent fill and build properties. Easy to sand.Grey, black or white.Kit includes: Pro Form Direct to Metal 2K Urethane High Build Primer 3.79L (

High Performance Foam Pads For Random Orbital and Gear Driven PolishersYellow (Fine). 150/180mm.  Per unit.The RUPES D-A Fine Foam Pad is the most hugely versatile pad in the RUPES BigFoot range. Pair with RUPES DA-FINE polishing compound to remove moderate to mild defects at an effective rate of defect removal, while creating a high-gloss finish on most paint systems. We love to think of this pad as almost a 'one-step' pad; as it's versatility can truly transform such as wide range of paintwork types; and is amazingly comfortable and easy to use.These new RUPES pads have a patented new design that has been designed and manufactured in Italy. This new design offers a lower profile pad height that effectively optimises the transfer of heat from the tool to the application surface (resulting in a more effective defect removal and higher quality finish); and also an all new contour profile which helps maintain safe machine operation near panel edges. These features combined improve the co

Pack of 25.
9mm Head16mm Long4mm HolePack of 25.
Head: 8mm.Length: 18mm.Hole: 5mm.Pack of 25.
Head: 10mm.Length: 30mm.Hole: 6mm.Pack of 25.
Pack of 25.
Pack of 25.
Pack of 25.
Pack of 25.
Pack of 25.
Pack of 25.
Compact and portable multi-functional welderThe CMI161 is designed as a compact and portable multi-functional welding machine, primarily for MIG welding but also for the MMA welding process; a modern single-phase 230V inverter machine suitable for workshop use and site welding operations.  Simple and user-friendly the CMI161 has a digital control panel with smart welding programmes. Turn one knob, then all other welding parameters are automatically set.The machine is single-phase (230 V) and has PFC – Power Factor Correction – to allow you to weld with a long mains cable or a generator as your power source. MIG with built-in intelligenceJust switch on, press and weld!....Select MIG directly on the digital control panel. Enter the wire/gas combination … CMI161 sets all other par

...

 Max buffer 75mm. 900W. 1500 - 4000 R.P.M.  1.8Kg.  For vertical polishing and small retouching operations.RUPES model RUFR32ES is a mini-polisher with ultra-small weight and dimensions, yet extremely powerful and high-performing.Features include electronic Soft Start-Up device for spreading the polishing pastes without wasting the product, and speed regulation with torque control.RUFR32ES is ideal for vertical polishing and retouching operations that require the use of a small tool. Also ideal for removal of enamels and glazes and for fine finishing work. Technical Specifications:   Polishing Pad Size Max. mm 75   Pad Thread M14   Power W 900   Speed Control √   Weight Kg 1.8   Revs R.P.M 1500-4000   Side Handle √  Recommended Pads:   Ø mm Interface Pad thread Code   75 Velcro M14 990.015  Downloads:   			RUPES Catalogue (6MB) 			Parts Diagram   
Thinking Centralised Dust Extraction?Think HE500. The High Performance RUPES 4Hp Turbine Centralised Dust Extraction System You Can Install 

iBrid Technology is now a reality!The Rupes Nano iBrid ushers in an era of innovation, flexibility and versatility, with the ability to easily polish places you never could before!  The new Rupes Bigfoot Nano iBrid does away with the decades of contorting your hand with a towel wrapped around it to attempt to reach those tiny little areas that a machine never could!  The functionality of a battery with no cord allows for even more ease of access and the versatility of easily snapping the corded power supply back on and forgoing the battery is incredible!  All of this is possible without interrupting the polishing process.   The dual power supply is joined by multi-action movements (both Dual Action or Rotary movement in a single unit) revolutionizes the approach to cleaning and detailing. The high performance motor saves energy, offers optimal performance, and produces a low environmental impact.Backed by the BigFoot family name, its pedigree is one of reliability and quality. Paired w

 1200W. Extremely quiet unit. 5Kg Dust bag capacity.  45L Container volume.RUS145EL is the latest portable energy/dust extraction unit from RUPES. This extremely quiet unit is suitable for extracting dust from sanding and various types of industrial work. Fitted with a  connection for an electrical tool.Features include: self-cleaning filter system, manual or automatic start-up options, speed control, liquid-level indicator, working-time memory and service-interval indicator for replacement of motor brushes, an electrostatic charge dispersion device, and pivotable front connector for hoses Ø 25 - 29 - 38 - 50mm.This unit includes a large dust and liquid container, generous rear space for cables and accessories, plus top-mounted ledge for accessories and tools. Technical Specifications:   Filter Class EN 60335-2-69 M   Power W 1x1200   Tool Power, max W 1x2400   Air Flow Rate m3/h 200   Vacuum Level mm/H©üO 2000   Weight Kg 13   Noise dB(A) 67   Bag Capacity Kg 5   Container Volume L 45

 Very good adhesion to metal. Good hardness with high elasticity. Minimal shrinkage. Good sanding propertiesPolyester resin with suitable liquid activator for manual laminating. In connection with glass mat it is used for car body repairs as well as camping caravans,car body repairs, boats and yachts.A three-component product for repairing large holes. It is a polyester resin with a properly selected accelerator. The cropped glass mat saves working time. The obtained coating has very good adhesion to the substrate.Designed for manual application, resistant to gasoline, greases, oils, diluted acids and bases.Do not apply on reactive primers, one-component acrylic and nitrocellulose products.    Downloads:   			Safety Data Sheet (425KB)   			Hardener Data Sheet (802KB) 			Technical Data Sheet (760KB)  
Two component, transparent high gloss clearcoat. Low VOC.Use with hardener #TMBH0.5.2 to 1 Mixing ratio.    Downloads for TMBGC1L:   			Safety Data Sheet 			Fact Sheet (869KB)  
 Waterproo

Wirbel is a leading European brand of affordable, high quality, industrial strength vacuum cleaners.  Compact and quiet. 1000W. 18L Capacity. 10m Cable length. 9Kg.The 829P model is a compact high performance, low noise wet and dry vacuum.With PP tank mounted on an ABS base with 4 rotating castors.Comes complete with metal wand, hose and cleaning head/attachments and 10m cable. Technical Specifications:   Rated Power W 1000   Capacity L 18   Vacuum m Bar 210   Air flow rate l/s 45   Cable length m 10   Dimensions cm 34x34x73   Weight Kg 9   Noise dB(A) 65  Downloads:     Use and Maintenance (80KB)  
Wirbel is a leading European brand of affordable, high quality, industrial strength vacuum cleaners.  Compact and quiet. 1000W. 18L Capacity. Electrical socket with auto/manual start for powertools. 10m Cable length. 9Kg.The 829I RE model is a compact high performance, low noise wet and dry vacuum with stainless steel tank mounted on an ABS base with 4 rotating castors.Also includes an elec